## Import the Library

In [1]:
from bs4 import BeautifulSoup
import requests

#### Download the web page containing the forecast.

In [2]:
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")

#### Create a BeautifulSoup class to parse the page.

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')

#### Find the div with id seven-day-forecast, and assign to seven_day

In [4]:
seven_day = soup.find(id="seven-day-forecast")

#### Inside seven_day, find each individual forecast item.

In [5]:
forecast_items = seven_day.find_all(class_="tombstone-container")

#### Extract and print the first forecast item.

In [6]:
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Today
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Today: Sunny, with a high near 56. Breezy, with a north wind around 25 mph, with gusts as high as 33 mph. " class="forecast-icon" src="newimages/medium/wind_few.png" title="Today: Sunny, with a high near 56. Breezy, with a north wind around 25 mph, with gusts as high as 33 mph. "/>
 </p>
 <p class="short-desc">
  Sunny and
  <br/>
  Breezy
 </p>
 <p class="temp temp-high">
  High: 56 °F
 </p>
</div>


#### We'll extract the name of the forecast item

In [8]:
period = tonight.find(class_ = "period-name").get_text()
short_Desc = tonight.find(class_ = "short-desc").get_text()
temp = tonight.find(class_ = "temp temp-high").get_text()

print(period)
print(short_Desc)
print(temp)

Today
Sunny andBreezy
High: 56 °F


#### we can extract the title attribute from the img tag

In [9]:
img = tonight.find('img')

In [10]:
img.prettify()

'<img alt="Today: Sunny, with a high near 56. Breezy, with a north wind around 25 mph, with gusts as high as 33 mph. " class="forecast-icon" src="newimages/medium/wind_few.png" title="Today: Sunny, with a high near 56. Breezy, with a north wind around 25 mph, with gusts as high as 33 mph. "/>\n'

In [11]:
desc =img['title']
print(desc)

Today: Sunny, with a high near 56. Breezy, with a north wind around 25 mph, with gusts as high as 33 mph. 


#### I use list comprehension to call the get_text method on each BeautifulSoup object

In [12]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Today',
 'Tonight',
 "NewYear'sDay",
 'TuesdayNight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight',
 'Friday']

In [13]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

print(short_descs)
print(temps)
print(descs)

['Sunny andBreezy', 'Clear andBreezy', 'Sunny', 'Mostly Clear', 'Sunny', 'Mostly Clear', 'Mostly Sunny', 'Partly Cloudy', 'Partly Sunny']
['High: 56 °F', 'Low: 42 °F', 'High: 54 °F', 'Low: 42 °F', 'High: 55 °F', 'Low: 43 °F', 'High: 56 °F', 'Low: 45 °F', 'High: 57 °F']
['Today: Sunny, with a high near 56. Breezy, with a north wind around 25 mph, with gusts as high as 33 mph. ', 'Tonight: Clear, with a low around 42. Breezy, with a northeast wind 19 to 24 mph decreasing to 13 to 18 mph after midnight. Winds could gust as high as 31 mph. ', "New Year's Day: Sunny, with a high near 54. North northeast wind 5 to 10 mph. ", 'Tuesday Night: Mostly clear, with a low around 42. Northeast wind 6 to 9 mph becoming south southeast in the evening. ', 'Wednesday: Sunny, with a high near 55. Southeast wind around 5 mph becoming north in the morning. ', 'Wednesday Night: Mostly clear, with a low around 43.', 'Thursday: Mostly sunny, with a high near 56.', 'Thursday Night: Partly cloudy, with a low ar

#### Combining our data into a Pandas Dataframe

In [14]:
import pandas as pd
weather = pd.DataFrame({
        "period": periods, 
        "short_desc": short_descs, 
        "temp": temps, 
        "desc":descs
    })
weather

,period,short_desc,temp,desc
0,Today,Sunny andBreezy,High: 56 °F,"Today: Sunny, with a high near 56. Breezy, wit..."
1,Tonight,Clear andBreezy,Low: 42 °F,"Tonight: Clear, with a low around 42. Breezy, ..."
2,NewYear'sDay,Sunny,High: 54 °F,"New Year's Day: Sunny, with a high near 54. No..."
3,TuesdayNight,Mostly Clear,Low: 42 °F,"Tuesday Night: Mostly clear, with a low around..."
4,Wednesday,Sunny,High: 55 °F,"Wednesday: Sunny, with a high near 55. Southea..."
5,WednesdayNight,Mostly Clear,Low: 43 °F,"Wednesday Night: Mostly clear, with a low arou..."
6,Thursday,Mostly Sunny,High: 56 °F,"Thursday: Mostly sunny, with a high near 56."
7,ThursdayNight,Partly Cloudy,Low: 45 °F,"Thursday Night: Partly cloudy, with a low arou..."
8,Friday,Partly Sunny,High: 57 °F,"Friday: Partly sunny, with a high near 57."


#### Use a regular expression and the Series.str.extract method to pull out the numeric temperature values:

In [15]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    56
1    42
2    54
3    42
4    55
5    43
6    56
7    45
8    57
Name: temp_num, dtype: object

In [16]:
weather["temp_num"].mean()

50.0

In [17]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0    False
1     True
2    False
3     True
4    False
5     True
6    False
7     True
8    False
Name: temp, dtype: bool

In [18]:
weather[is_night]

,period,short_desc,temp,desc,temp_num,is_night
1,Tonight,Clear andBreezy,Low: 42 °F,"Tonight: Clear, with a low around 42. Breezy, ...",42,True
3,TuesdayNight,Mostly Clear,Low: 42 °F,"Tuesday Night: Mostly clear, with a low around...",42,True
5,WednesdayNight,Mostly Clear,Low: 43 °F,"Wednesday Night: Mostly clear, with a low arou...",43,True
7,ThursdayNight,Partly Cloudy,Low: 45 °F,"Thursday Night: Partly cloudy, with a low arou...",45,True
